# BOKEH Visualization notebook
The notebook contains two visualizations featuring 
1. Slider
2. Hover tool

It takes three datasets for from gapminder.

1. [Total energy generation](http://docs.google.com/spreadsheet/pub?key=pyj6tScZqmEehRG-9mMHYdg&output=xlsx)
2. [Total energy consumption](http://docs.google.com/spreadsheet/pub?key=tEu78F4acf0u6MRyhg5-9qQ&output=xlsx)
3. [GDP per capita](http://docs.google.com/spreadsheet/pub?key=0AkBd6lyS3EmpdHo5S0J6ekhVOF9QaVhod05QSGV4T3c&output=xlsx)

In [61]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import CustomJS, Slider, HoverTool,ColumnDataSource
from bokeh.layouts import row
from bokeh.charts import Scatter

# Data Preprocessing
The data from gapminder has missing values thus the cell below performs inner join operations to get rid of the noise. Thus the data remaining is for 64 countries after cleaning up.

In [62]:
#using pandas to read all three datasets from excel files and taking an inner join for rows and columns to maintain consistency of data along all three datasets
produceTotal = pd.read_excel("Electricity Generation.xls.xlsx")
produceTotal=produceTotal.set_index('Electricity generation total (kilowatt-hours)')
country_list=produceTotal.index.tolist()
year_list=produceTotal.columns.tolist()
year_list=[str(i) for i in year_list]

consumeTotalRaw = pd.read_excel("Indicator_Electricity consumption total.xlsx")
consumeTotalRaw=consumeTotalRaw.set_index('Electricity consumption, total (kWh)')
consumeTotalRaw=consumeTotalRaw.loc[:,year_list]
consumeTotal=consumeTotalRaw.loc[country_list]

gdpRaw = pd.read_excel("gdp_data.xlsx")
gdpRaw=gdpRaw.set_index('GDP (constant 2000 US$)')
gdp=gdpRaw.loc[country_list]

# Relation between energy consumption and energy production

The slider defines the year in which the relationship is shown. Hovering on the data points displays the name of the country and the values of the corresponding x and y coordinates.

**NOTE : Please run the cell below individually for the slider function**

In [65]:
#sending the output to the notebook
output_notebook()

#creating a data dictionary to be passed to the ColumnDataSource. passing country list for hover tool
data = {
    'x_values' : produceTotal[1990],
    'y_values' : consumeTotal['1990'],
    'country' : country_list
}


source=ColumnDataSource(data=data)

#Setting up hover tooltips
hover = HoverTool(tooltips=[
    ("Country", "@country"),
    ("(x,y)", "($x, $y)")
])

#defining the figure object then adding a circle plot to it
fig = figure(plot_width=500, plot_height=500, tools=[hover,'wheel_zoom','box_zoom','box_select','reset'],title="Energy consumption vs. Production",x_axis_type="log",y_axis_type="log")
plt = fig.circle(x='x_values', y='y_values',size=10,color='blue',alpha=0.3,source=source)
fig.xaxis.axis_label= "Total Energy Production in kWh (Log scale)"
fig.yaxis.axis_label= "Total Energy Consumption in kWh (Log scale)"

#python function for updating the plot. Passing the slider value (year) as the argument
def update_year(year):
    plt.data_source.data['x_values']=produceTotal[year]
    plt.data_source.data['y_values']=consumeTotal[str(year)]
    push_notebook(handle=bokeh_handle)  #push_notebook takes care of updating the python notebook view

#javaScript callback function to invoke the python update function constantly
callback = CustomJS(code="""
if (IPython.notebook.kernel !== undefined) {
    var kernel = IPython.notebook.kernel;
    cmd = "update_year(" + cb_obj.value + ")";
    kernel.execute(cmd, {}, {});}
""")

#creating slider object instance and passing the callback function to it
slider = Slider(start=1990, 
                end=2008,
                value=1990,
                step=1,
                title="Year",
                callback=callback)
#Displaying the plot and the slider in a row layout
bokeh_handle = show(row(fig, slider), notebook_handle=True)

Loading BokehJS ...

# Relation between energy production and gdp

The slider defines the year in which the relationship is shown. Hovering on the data points displays the name of the country and the values of the corresponding x and y coordinates.

**NOTE : Please run the cell below individually for the slider function**

In [66]:
output_notebook()
data1 = {
    'x_val' : produceTotal[1990],
    'y_val' : gdp[1990],
    'country' : country_list
}

source1=ColumnDataSource(data=data1)
hover1 = HoverTool(tooltips=[
    ("Country", "@country"),
    ("(x,y)", "($x, $y)")
])
fig1 = figure(plot_width=500, plot_height=500, tools=[hover1,'wheel_zoom','box_zoom','box_select','reset'],title="Relationship between energy production and GDP",x_axis_type="log",y_axis_type="log")
plt1 = fig1.circle(x='x_val', y='y_val',size=10,color='firebrick',alpha=0.3,source=source1)
fig1.xaxis.axis_label= "Total Energy Production in kWh (Log scale)"
fig1.yaxis.axis_label= "Total GDP in USD (Log scale)"
def update_year1(year):
    plt1.data_source.data['x_val']=produceTotal[year]
    plt1.data_source.data['y_val']=gdp[year]
    push_notebook(handle=bokeh_handle)  

callback1 = CustomJS(code="""
    var kernel = IPython.notebook.kernel;
    cmd = "update_year1(" + cb_obj.value + ")";
    kernel.execute(cmd, {}, {});
""")

slider1 = Slider(start=1990, 
                end=2008,
                value=1990,
                step=1,
                title="Year")
slider1.js_on_change('value', callback1)
bokeh_handle = show(row(fig1, slider1), notebook_handle=True)

Loading BokehJS ...